# Videogames Analysis

Formatos de revisor

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Correcto
</div>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Observaciones y recomendaciones
</div>

<div class="alert alert-block alert-danger">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Necesita correcciones
</div>

<div class="alert alert-block alert-info">
<b>Respuesta del estudiante.</b> <a class="tocSkip"></a>

Respuesta del estudiante
</div>

*Start by importing libraries*

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stats

*Bring the dataset and save it in in a variable. I make sure the column name's are lower case*

In [2]:
url = "https://raw.githubusercontent.com/leobabel/ProyectoIntegrado1/main/games.csv"
df = pd.read_csv(url)
df.columns = df.columns.str.lower()
print(df.head())
print()
print(df.info())


                       name platform  year_of_release         genre  na_sales  \
0                Wii Sports      Wii           2006.0        Sports     41.36   
1         Super Mario Bros.      NES           1985.0      Platform     29.08   
2            Mario Kart Wii      Wii           2008.0        Racing     15.68   
3         Wii Sports Resort      Wii           2009.0        Sports     15.61   
4  Pokemon Red/Pokemon Blue       GB           1996.0  Role-Playing     11.27   

   eu_sales  jp_sales  other_sales  critic_score user_score rating  
0     28.96      3.77         8.45          76.0          8      E  
1      3.58      6.81         0.77           NaN        NaN    NaN  
2     12.76      3.79         3.29          82.0        8.3      E  
3     10.93      3.28         2.95          80.0          8      E  
4      8.89     10.22         1.00           NaN        NaN    NaN  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11

## Data Prep

*Let's count the null values*

In [3]:
print(df.isnull().sum())

name                  2
platform              0
year_of_release     269
genre                 2
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       8578
user_score         6701
rating             6766
dtype: int64


*Let's visualize null values from the column "name"; we can observe that these two indexes (659 and 1422) also contain the null values from the column "genre".*

In [4]:
print(df[df['name'].isnull()])
print('_'*75)
print(df[df['genre'].isnull()])

      name platform  year_of_release genre  na_sales  eu_sales  jp_sales  \
659    NaN      GEN           1993.0   NaN      1.78      0.53      0.00   
14244  NaN      GEN           1993.0   NaN      0.00      0.00      0.03   

       other_sales  critic_score user_score rating  
659           0.08           NaN        NaN    NaN  
14244         0.00           NaN        NaN    NaN  
___________________________________________________________________________
      name platform  year_of_release genre  na_sales  eu_sales  jp_sales  \
659    NaN      GEN           1993.0   NaN      1.78      0.53      0.00   
14244  NaN      GEN           1993.0   NaN      0.00      0.00      0.03   

       other_sales  critic_score user_score rating  
659           0.08           NaN        NaN    NaN  
14244         0.00           NaN        NaN    NaN  


### Actions regarding null values and their justification:

1.- Removal of entries at indexes 659 and 14244: These two indexes exclusively contain null values in the 'name' and 'genre' columns.

2.- Elimination of 269 null values from the 'year_of_release' column:

These deletions account for only 1.63% of the total records. Consequently, their removal will not significantly affect the overall analysis.

3.- I'm going to convert the "user_score" column from object to float64. 

In [5]:
# Alright, go away, mf'ers
df.dropna(subset=['name', 'genre', 'year_of_release'], inplace=True)
print(df.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 16444 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16444 non-null  object 
 1   platform         16444 non-null  object 
 2   year_of_release  16444 non-null  float64
 3   genre            16444 non-null  object 
 4   na_sales         16444 non-null  float64
 5   eu_sales         16444 non-null  float64
 6   jp_sales         16444 non-null  float64
 7   other_sales      16444 non-null  float64
 8   critic_score     7983 non-null   float64
 9   user_score       9839 non-null   object 
 10  rating           9768 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.5+ MB
None


Let's change 'year_of_release' data type from float64 to something more useful.

In [6]:
# Convert 'year_of_release' to datetime keeping only the year
df['year_of_release'] = pd.to_datetime(df['year_of_release'], format='%Y', errors='coerce').dt.year

# Verify the changes
print(df.head())
print(df.info())


                       name platform  year_of_release         genre  na_sales  \
0                Wii Sports      Wii             2006        Sports     41.36   
1         Super Mario Bros.      NES             1985      Platform     29.08   
2            Mario Kart Wii      Wii             2008        Racing     15.68   
3         Wii Sports Resort      Wii             2009        Sports     15.61   
4  Pokemon Red/Pokemon Blue       GB             1996  Role-Playing     11.27   

   eu_sales  jp_sales  other_sales  critic_score user_score rating  
0     28.96      3.77         8.45          76.0          8      E  
1      3.58      6.81         0.77           NaN        NaN    NaN  
2     12.76      3.79         3.29          82.0        8.3      E  
3     10.93      3.28         2.95          80.0          8      E  
4      8.89     10.22         1.00           NaN        NaN    NaN  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16444 entries, 0 to 16714
Data columns (total 11 

Let's Replace 'tbd' with NaN in the 'user_score' column, because we're going to change its data type from object to float64. Then, we convert 'user_score' column from object to float64 in order to make statistical calculations further

In [7]:
df['user_score'] = df['user_score'].replace('tbd', np.nan)
df['user_score'] = df['user_score'].astype('float64')

# Verify the changes
print(df.head())
print(df.info())

                       name platform  year_of_release         genre  na_sales  \
0                Wii Sports      Wii             2006        Sports     41.36   
1         Super Mario Bros.      NES             1985      Platform     29.08   
2            Mario Kart Wii      Wii             2008        Racing     15.68   
3         Wii Sports Resort      Wii             2009        Sports     15.61   
4  Pokemon Red/Pokemon Blue       GB             1996  Role-Playing     11.27   

   eu_sales  jp_sales  other_sales  critic_score  user_score rating  
0     28.96      3.77         8.45          76.0         8.0      E  
1      3.58      6.81         0.77           NaN         NaN    NaN  
2     12.76      3.79         3.29          82.0         8.3      E  
3     10.93      3.28         2.95          80.0         8.0      E  
4      8.89     10.22         1.00           NaN         NaN    NaN  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16444 entries, 0 to 16714
Data columns (tot

Let's check the unique values in 'rating' colum in order to get familiarized with the titles

In [8]:
unique_ratings = df['rating'].unique()
for rating in unique_ratings:
    print(rating)

E
nan
M
T
E10+
K-A
AO
EC
RP


Alright, so, these ratings correspond to the ESRB (Entertainment Software Rating Board) ratings for video games. Here's what each rating means:

- <b>E (Everyone):</b> Suitable for all ages. Content is generally suitable for all ages. May contain minimal cartoon, fantasy, or mild violence and/or infrequent use of mild language.

- <b>EC (Early Childhood):</b> Suitable for young children. Content is intended for young children and contains no material that parents would find inappropriate.

- <b>E10+ (Everyone 10 and older):</b> Suitable for ages 10 and older. Content is generally suitable for ages 10 and up. May contain more cartoon, fantasy, or mild violence, mild language, and/or minimal suggestive themes.

- <b>K-A (Kids to Adults):</b> An early rating used by the ESRB before 1998, which was replaced by the "E" rating. Suitable for ages 6 and older.

- <b>T (Teen):</b> Suitable for ages 13 and older. Content may include violence, suggestive themes, crude humor, minimal blood, simulated gambling, and/or infrequent use of strong language.

- <b>M (Mature):</b> Suitable for ages 17 and older. May contain intense violence, blood and gore, sexual content, and/or strong language.

- <b>AO (Adults Only):</b> Suitable only for adults ages 18 and older. May include prolonged scenes of intense violence, graphic sexual content, and/or gambling with real currency.

- <b>RP (Rating Pending):</b> This is a placeholder used for games that have been submitted to the ESRB and are awaiting a final rating.

These ratings help consumers, especially parents, make informed choices about the video games they purchase.

In [9]:
# Obtener los valores únicos en la columna 'rating'
unique_ratings = df['rating'].unique()

# Obtener el total de registros en el DataFrame
total_records = len(df)

# Iterar sobre cada rating único
for rating in unique_ratings:
    # Contar el número de registros con el rating actual
    rating_count = df[df['rating'] == rating].shape[0]
    
    # Calcular el porcentaje de registros que representan
    percentage = (rating_count / total_records) * 100
    
    # Imprimir el rating, el número de registros y el porcentaje
    print(f"Rating: {rating}, Registros: {rating_count}, Porcentaje: {percentage:.2f}%")


Rating: E, Registros: 3921, Porcentaje: 23.84%
Rating: nan, Registros: 0, Porcentaje: 0.00%
Rating: M, Registros: 1536, Porcentaje: 9.34%
Rating: T, Registros: 2905, Porcentaje: 17.67%
Rating: E10+, Registros: 1393, Porcentaje: 8.47%
Rating: K-A, Registros: 3, Porcentaje: 0.02%
Rating: AO, Registros: 1, Porcentaje: 0.01%
Rating: EC, Registros: 8, Porcentaje: 0.05%
Rating: RP, Registros: 1, Porcentaje: 0.01%


In [10]:
# Get unique values in the 'platform' column
unique_platforms = df['platform'].unique()
for platform in unique_platforms:
    print(platform)
print()
# Count the number of unique platforms
num_unique_platforms = df['platform'].nunique()
print(f"Number of platforms: {num_unique_platforms}")


Wii
NES
GB
DS
X360
PS3
PS2
SNES
GBA
PS4
3DS
N64
PS
XB
PC
2600
PSP
XOne
WiiU
GC
GEN
DC
PSV
SAT
SCD
WS
NG
TG16
3DO
GG
PCFX

Number of platforms: 31


Alright, now we'd like to know many genres we do have and which genres this dataframe includes:

In [11]:
# Obtener los valores únicos en la columna 'genre'
unique_genres = df['genre'].unique()

# Crear un diccionario para almacenar los conteos de cada género
genre_counts = {}

# Contar el número total de registros
total_records = len(df)

# Iterar sobre cada género único y contar el número de registros
for genre in unique_genres:
    genre_count = df[df['genre'] == genre].shape[0]
    genre_counts[genre] = genre_count

# Ordenar los géneros por conteo en orden descendente
sorted_genres = sorted(genre_counts.items(), key=lambda x: x[1], reverse=True)

# Imprimir los géneros, sus conteos y porcentajes en orden descendente
for genre, count in sorted_genres:
    percentage = (count / total_records) * 100
    print(f"Género: {genre} | Registros: {count} | Porcentaje: {percentage:.2f}%")

# Contar el número de géneros únicos
num_unique_genres = df['genre'].nunique()
print(f"\nNúmero de géneros: {num_unique_genres}")

Género: Action | Registros: 3307 | Porcentaje: 20.11%
Género: Sports | Registros: 2306 | Porcentaje: 14.02%
Género: Misc | Registros: 1721 | Porcentaje: 10.47%
Género: Role-Playing | Registros: 1481 | Porcentaje: 9.01%
Género: Shooter | Registros: 1296 | Porcentaje: 7.88%
Género: Adventure | Registros: 1293 | Porcentaje: 7.86%
Género: Racing | Registros: 1226 | Porcentaje: 7.46%
Género: Platform | Registros: 878 | Porcentaje: 5.34%
Género: Simulation | Registros: 857 | Porcentaje: 5.21%
Género: Fighting | Registros: 837 | Porcentaje: 5.09%
Género: Strategy | Registros: 673 | Porcentaje: 4.09%
Género: Puzzle | Registros: 569 | Porcentaje: 3.46%

Número de géneros: 12


In [12]:
# Find the minimum and maximum year in the 'year_of_release' column
min_year = df['year_of_release'].min()
max_year = df['year_of_release'].max()

# Calculate the span of time
time_span = max_year - min_year

print(f"The minimum year is: {min_year}")
print(f"The maximum year is: {max_year}")
print(f"The span of time in the dataframe is: {time_span} years")


The minimum year is: 1980
The maximum year is: 2016
The span of time in the dataframe is: 36 years


Let's sum all sales and place those total sales un a new column

In [16]:
# Sum sales from 'na_sales', 'eu_sales', 'jp_sales', 'other_sales'
df['total_sales'] = df[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum(axis=1)

# Insert the new column 'total_sales' after 'other_sales'
other_sales_index = df.columns.get_loc('other_sales')
df.insert(other_sales_index + 1, 'total_sales', df.pop('total_sales'))

# Display the modified dataframe
print(df.head())


                       name platform  year_of_release         genre  na_sales  \
0                Wii Sports      Wii             2006        Sports     41.36   
1         Super Mario Bros.      NES             1985      Platform     29.08   
2            Mario Kart Wii      Wii             2008        Racing     15.68   
3         Wii Sports Resort      Wii             2009        Sports     15.61   
4  Pokemon Red/Pokemon Blue       GB             1996  Role-Playing     11.27   

   eu_sales  jp_sales  other_sales  total_sales  critic_score  user_score  \
0     28.96      3.77         8.45        82.54          76.0         8.0   
1      3.58      6.81         0.77        40.24           NaN         NaN   
2     12.76      3.79         3.29        35.52          82.0         8.3   
3     10.93      3.28         2.95        32.77          80.0         8.0   
4      8.89     10.22         1.00        31.38           NaN         NaN   

  rating  
0      E  
1    NaN  
2      E  
3     

In [18]:
# Calculate statistics for 'critic_score'
min_cs = df['critic_score'].min()
max_cs = df['critic_score'].max()
average_cs = df['critic_score'].mean()
median_cs = df['critic_score'].median()
std_dev_cs = df['critic_score'].std()
q1_cs = df['critic_score'].quantile(0.25)
q3_cs = df['critic_score'].quantile(0.75)
count_cs = df['critic_score'].count()
mode_cs = df['critic_score'].mode()[0] if not df['critic_score'].mode().empty else 'No mode'

# Get the games with the lowest and highest critic scores
game_with_lowest_cs = df.loc[df['critic_score'].idxmin()]
game_with_highest_cs = df.loc[df['critic_score'].idxmax()]

# Print results
print(f"The game with the lowest critic score:\n{game_with_lowest_cs}\n")
print(f"The game with the highest critic score:\n{game_with_highest_cs}\n")
print(f"Minimum critic score: {min_cs}")
print(f"Maximum critic score: {max_cs}")
print(f"Average critic score: {average_cs:.2f}")
print(f"Median critic score: {median_cs}")
print(f"Standard deviation of critic scores: {std_dev_cs:.2f}")
print(f"25th percentile (Q1) of critic scores: {q1_cs}")
print(f"75th percentile (Q3) of critic scores: {q3_cs}")
print(f"Count of non-null critic scores: {count_cs}")
print(f"Mode of critic scores: {mode_cs}")

The game with the lowest critic score:
name               Ride to Hell
platform                    PS3
year_of_release            2013
genre                    Racing
na_sales                   0.02
eu_sales                   0.01
jp_sales                    0.0
other_sales                0.01
total_sales                0.04
critic_score               13.0
user_score                  1.0
rating                        M
Name: 13689, dtype: object

The game with the highest critic score:
name               Grand Theft Auto IV
platform                          X360
year_of_release                   2008
genre                           Action
na_sales                          6.76
eu_sales                          3.07
jp_sales                          0.14
other_sales                       1.03
total_sales                       11.0
critic_score                      98.0
user_score                         7.9
rating                               M
Name: 51, dtype: object

Minimum critic s

In [19]:
# Calculate statistics for 'user_score'
min_us = df['user_score'].min()
max_us = df['user_score'].max()
average_us = df['user_score'].mean()
median_us = df['user_score'].median()
std_dev_us = df['user_score'].std()
q1_us = df['user_score'].quantile(0.25)
q3_us = df['user_score'].quantile(0.75)
count_us = df['user_score'].count()
mode_us = df['user_score'].mode()[0] if not df['user_score'].mode().empty else 'No mode'

# Get the games with the lowest and highest critic scores
game_with_lowest_us = df.loc[df['user_score'].idxmin()]
game_with_highest_us = df.loc[df['user_score'].idxmax()]

# Print results
print(f"The game with the lowest user score:\n{game_with_lowest_us}\n")
print(f"The game with the highest user score:\n{game_with_highest_us}\n")
print(f"Minimum user score: {min_us}")
print(f"Maximum user score: {max_us}")
print(f"Average user score: {average_us:.2f}")
print(f"Median user score: {median_us}")
print(f"Standard deviation of user score: {std_dev_us:.2f}")
print(f"25th percentile (Q1) of user score: {q1_us}")
print(f"75th percentile (Q3) of user score: {q3_us}")
print(f"Count of non-null user score: {count_us}")
print(f"Mode of user score: {mode_us}")

The game with the lowest user score:
name               My Little Pony: Pinkie Pie's Party
platform                                           DS
year_of_release                                  2008
genre                                       Adventure
na_sales                                         0.66
eu_sales                                          0.0
jp_sales                                          0.0
other_sales                                      0.06
total_sales                                      0.72
critic_score                                      NaN
user_score                                        0.0
rating                                              E
Name: 2835, dtype: object

The game with the highest user score:
name               Breath of Fire III
platform                          PSP
year_of_release                  2005
genre                    Role-Playing
na_sales                          0.0
eu_sales                         0.02
jp_sales              

In [15]:
sales_columns = ['na_sales', 'eu_sales', 'jp_sales', 'other_sales']
sales_statistics = df[sales_columns].describe()
print(sales_statistics)


           na_sales      eu_sales      jp_sales   other_sales
count  16444.000000  16444.000000  16444.000000  16444.000000
mean       0.264012      0.145930      0.078487      0.047594
std        0.818378      0.506716      0.311100      0.188005
min        0.000000      0.000000      0.000000      0.000000
25%        0.000000      0.000000      0.000000      0.000000
50%        0.080000      0.020000      0.000000      0.010000
75%        0.240000      0.110000      0.040000      0.030000
max       41.360000     28.960000     10.220000     10.570000
